In [13]:
import asyncio
import websockets
import json
import requests
import ssl
import certifi
import threading

First, we implement a function to get the initial order book (limit first 1000 records)snapshot from the REST API. It returns a json file with 3 keys: 'lastUpdateId', 'bids' and 'asks'.

In [14]:
def get_order_book_snapshot(symbol):
    url = f"https://api.binance.com/api/v3/depth"
    params = {"symbol": symbol, "limit": 1000}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to retrieve initial order book snapshot.")
        return None

In [15]:
order_book = get_order_book_snapshot('BTCUSDT')

In [16]:
order_book.keys()

dict_keys(['lastUpdateId', 'bids', 'asks'])

In [17]:
# two subelement in every bid or ask element, first is price and second is quantity.
print(order_book['bids'][0])
print(order_book['asks'][0])

['71273.44000000', '4.38861000']
['71273.45000000', '0.00432000']


Next, we implement a function to update the local order book based on incoming events

In [18]:
def update_order_book(order_book, bids, asks):
    for price, quantity in bids:
        if quantity == '0.00000000':
            order_book['bids'] = [b for b in order_book['bids'] if b[0] != price]
        else:
            order_book['bids'] = [b for b in order_book['bids'] if b[0] != price]
            order_book['bids'].append([price, quantity])

    for price, quantity in asks:
        if quantity == '0.00000000':
            order_book['asks'] = [a for a in order_book['asks'] if a[0] != price]
        else:
            order_book['asks'] = [a for a in order_book['asks'] if a[0] != price]
            order_book['asks'].append([price, quantity])

    order_book['bids'] = sorted(order_book['bids'], key=lambda x: float(x[0]), reverse=True)[:5]
    order_book['asks'] = sorted(order_book['asks'], key=lambda x: float(x[0]))[:5]

Then we implement a function to display the order book.

In [19]:
def display_order_book(order_book):
    print("\nOrder Book:")
    print("Bids:")
    for bid in order_book['bids']:
        print(f"Price: {bid[0]}, Quantity: {bid[1]}")
    
    print("\nAsks:")
    for ask in order_book['asks']:
        print(f"Price: {ask[0]}, Quantity: {ask[1]}")

Followed by the main function to handle WebSocket connection and order book updates

In [22]:
async def track_order_book(symbol):
    ws_url = f"wss://stream.binance.com:9443/ws/{symbol.lower()}@depth"
    
    # Get the initial snapshot and lastUpdateId
    order_book = get_order_book_snapshot(symbol)
    if order_book is None:
        return

    last_update_id = order_book['lastUpdateId']
    # display_order_book(order_book)

    # SSL context using certifi for certificate validation
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    ssl_context.load_verify_locations(certifi.where())

    async with websockets.connect(ws_url, ssl=ssl_context) as websocket:
        print("Connected to Binance WebSocket.")

        buffer = []
        while True:
            try:
                message = await websocket.recv()
                data = json.loads(message)
                # print(data)
                buffer.append(data)
            
                if data['u'] > last_update_id: 
                    break

            except Exception as e:
                print(f"Error while receiving message: {e}")
                break
        # print(buffer[0].keys())
        
        # Process buffered events
        for event in buffer:
            if event['u'] <= last_update_id: # Drop any event where u is <= lastUpdateId in the snapshot
                continue
            # The first processed event should have U <= lastUpdateId+1 AND u >= lastUpdateId+1
            if event['U'] <= last_update_id + 1 and event['u'] >= last_update_id + 1:
                update_order_book(order_book, event['b'], event['a'])
                last_update_id = event['u']
                display_order_book(order_book)

        while True:
            try:
                message = await websocket.recv()
                data = json.loads(message)
  
                if data['U'] == last_update_id + 1: # each new event's U should be equal to the previous event's u+1.
                    update_order_book(order_book, data['b'], data['a'])
                    last_update_id = data['u']
                    display_order_book(order_book)
                else:
                    print("Out of order event detected.")
                    break

            except Exception as e:
                print(f"Error while receiving message: {e}")
                break
    
if __name__ == "__main__":
    try:
        while True:
            symbol = input("Enter the Binance symbol: ").upper() # BTCUSDT
            await track_order_book(symbol)
    except asyncio.CancelledError:
        print("Process interrupted by user!")

Enter the Binance symbol: BTCUSDT
Connected to Binance WebSocket.
Out of order event detected.
Enter the Binance symbol: BTCUSDT
Connected to Binance WebSocket.

Order Book:
Bids:
Price: 71269.99000000, Quantity: 3.16245000
Price: 71269.98000000, Quantity: 0.00010000
Price: 71269.97000000, Quantity: 0.00018000
Price: 71269.96000000, Quantity: 0.06923000
Price: 71269.95000000, Quantity: 0.30291000

Asks:
Price: 71270.00000000, Quantity: 2.17837000
Price: 71270.02000000, Quantity: 0.01299000
Price: 71270.03000000, Quantity: 0.00010000
Price: 71270.04000000, Quantity: 0.00010000
Price: 71270.06000000, Quantity: 0.00010000

Order Book:
Bids:
Price: 71269.99000000, Quantity: 2.23564000
Price: 71269.98000000, Quantity: 0.00010000
Price: 71269.97000000, Quantity: 0.00018000
Price: 71269.96000000, Quantity: 0.06923000
Price: 71269.95000000, Quantity: 0.30334000

Asks:
Price: 71270.00000000, Quantity: 1.99305000
Price: 71270.02000000, Quantity: 0.01299000
Price: 71270.03000000, Quantity: 0.0001

Please note that this the cell can be interrupt with cancel button in the jupyter notebook instead of Ctrl+C, I did not managed to implement this interrupt function and I think I should note spend too much time on this because it is not the main focus of this task.

In [23]:
# Format of a data element in buffer list:
{'e': 'depthUpdate', 
 'E': 1730183523014, 
 's': 'BTCUSDT', 
 'U': 53552400692, 
 'u': 53552400785, 
 'b': [['71112.00000000', '1.79678000'], ['71111.96000000', '0.00010000'], ['71111.17000000', '0.19518000'], 
       ['71110.55000000', '0.28286000'], ['71110.30000000', '0.03034000'], ['71109.34000000', '0.00000000'], 
       ['71101.77000000', '0.00000000'], ['71099.64000000', '0.15669000'], ['71099.56000000', '0.00000000'], 
       ['71084.12000000', '0.00000000'], ['71083.90000000', '0.00000000'], ['71083.86000000', '0.00000000'], 
       ['71083.80000000', '0.00675000'], ['71083.59000000', '0.00000000'], ['71081.42000000', '0.00134000'], 
       ['71046.49000000', '0.00740000'], ['71024.14000000', '0.00000000'], ['71012.00000000', '0.13316000'], 
       ['71000.60000000', '0.00000000'], ['64000.00000000', '87.15291000'], ['60000.00000000', '177.04054000'], 
       ['33000.00000000', '5.98283000']], 
 'a': [['71112.01000000', '3.62047000'], ['71112.05000000', '0.00032000'],['71112.09000000', '0.52273000'], 
       ['71112.76000000', '0.00000000'], ['71120.00000000', '0.26303000'], ['71120.90000000', '0.00000000'], 
       ['71124.82000000', '0.00000000'], ['71126.84000000', '0.70360000'], ['71143.51000000', '0.04005000'], 
       ['71143.64000000', '0.00000000'], ['71143.85000000', '0.00000000'], ['71143.98000000', '0.00000000'], 
       ['71146.36000000', '0.01125000'], ['71147.26000000', '0.00000000'], ['71177.12000000', '0.00010000'], 
       ['71182.70000000', '0.00000000'], ['71212.00000000', '0.02720000'], ['71216.67000000', '0.49598000'], 
       ['71219.33000000', '0.00000000'], ['72468.78000000', '0.06903000'], ['72613.58000000', '0.06903000'], 
       ['75000.00000000', '253.82226300']]}

{'e': 'depthUpdate',
 'E': 1730183523014,
 's': 'BTCUSDT',
 'U': 53552400692,
 'u': 53552400785,
 'b': [['71112.00000000', '1.79678000'],
  ['71111.96000000', '0.00010000'],
  ['71111.17000000', '0.19518000'],
  ['71110.55000000', '0.28286000'],
  ['71110.30000000', '0.03034000'],
  ['71109.34000000', '0.00000000'],
  ['71101.77000000', '0.00000000'],
  ['71099.64000000', '0.15669000'],
  ['71099.56000000', '0.00000000'],
  ['71084.12000000', '0.00000000'],
  ['71083.90000000', '0.00000000'],
  ['71083.86000000', '0.00000000'],
  ['71083.80000000', '0.00675000'],
  ['71083.59000000', '0.00000000'],
  ['71081.42000000', '0.00134000'],
  ['71046.49000000', '0.00740000'],
  ['71024.14000000', '0.00000000'],
  ['71012.00000000', '0.13316000'],
  ['71000.60000000', '0.00000000'],
  ['64000.00000000', '87.15291000'],
  ['60000.00000000', '177.04054000'],
  ['33000.00000000', '5.98283000']],
 'a': [['71112.01000000', '3.62047000'],
  ['71112.05000000', '0.00032000'],
  ['71112.09000000', '0.5